In [1]:
import os
#os.environ["PYTORCH_ENABLE_MPS_FALLBACK"] = "1"

In [2]:
import torch
from transformers import GPTNeoForCausalLM, GPT2Tokenizer
from haystack.document_stores import InMemoryDocumentStore
from haystack.nodes import DensePassageRetriever, FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.schema import Document  # Corrigir import do Document

In [3]:
if torch.backends.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")

# Forçar o uso da CPU, independentemente de MPS estar disponível
device = torch.device("cpu")

In [4]:
# 1. Criar um armazenamento de documentos
document_store = InMemoryDocumentStore()

In [5]:
# 2. Indexar documentos (exemplo simples)
documents = [
    {"content": "A linguagem Python é popular para machine learning."},
    {"content": "O framework Haystack é usado para criar pipelines de NLP."},
    {"content": "GPT é um modelo de linguagem desenvolvido pela OpenAI."},
]

In [6]:
# Adiciona documentos ao armazenamento
document_store.write_documents(documents)

In [7]:
# 3. Configurar um recuperador DPR (Dense Passage Retriever)
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base"
)

/home/flavio/Dev/myenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [8]:
# 4. Atualizar o armazenamento de documentos com embeddings do retriever
document_store.update_embeddings(retriever)

Documents Processed: 10000 docs [00:00, 16318.17 docs/s]     


In [9]:
# 5. Criar um pipeline de perguntas e respostas
reader = FARMReader(model_name_or_path="pierreguillou/bert-large-cased-squad-v1.1-portuguese")
qa_pipeline = ExtractiveQAPipeline(reader=reader, retriever=retriever)

In [10]:
# Carregar o modelo e o tokenizer do GPT-2 (ou GPT-Neo para um modelo maior)
model_name = "EleutherAI/gpt-neo-125M"  # Um modelo adequado para várias línguas, incluindo português
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPTNeoForCausalLM.from_pretrained(model_name).to(torch.device(device))  # Forçando uso da CPU

In [11]:
# Função para verificar saudações e outras respostas simples
def predefined_responses(user_input):
    respostas = {
        "oi": "Oi, tudo bem?",
        "olá": "Olá! Como posso te ajudar?",
        "bom dia": "Bom dia! Como você está?",
        "boa tarde": "Boa tarde! Em que posso ajudar?",
        "boa noite": "Boa noite! O que você gostaria de saber?"
    }
    
    # Verificar se a pergunta do usuário está nas saudações predefinidas
    user_input_lower = user_input.lower()
    if user_input_lower in respostas:
        return respostas[user_input_lower]
    return None

In [12]:
# Função para geração de texto usando GPT-2 local
def generate_response(context, user_query, max_length=100):
    # Verificar saudações simples
    predefined = predefined_responses(user_query)
    if predefined:
        return predefined
    
    prompt = f"Pergunta do usuário: {user_query}\n\nContexto: {context}\n\nResposta da IA:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True).to(device)

    # Reduzir o comprimento máximo
    max_length = 100  # Definir um limite razoável para a resposta

    # Geração de texto usando o modelo local
    with torch.no_grad():
        outputs = model.generate(
            inputs["input_ids"],
            max_length=max_length,
            repetition_penalty=1.5,  # Penalidade para evitar repetições
            no_repeat_ngram_size=2,  # Evita repetições de bigramas (pares de palavras)
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

In [13]:
# Função para recuperação de documentos
def retrieve_documents(query):
    results = qa_pipeline.run(query=query, params={"Retriever": {"top_k": 3}, "Reader": {"top_k": 1}})
    return results['answers'][0].answer

In [14]:
# Função principal do chatbot usando RAG com LLM local
def chatbot(user_query):
    # Passo 1: Recuperar contexto relevante
    retrieved_context = retrieve_documents(user_query)
    
    # Passo 2: Gerar resposta com base no contexto recuperado
    response = generate_response(retrieved_context, user_query)
    
    return response

In [15]:
# Exemplo de interação com o chatbot
while True:
    user_input = input("Pergunta: ")
    if user_input.lower() in ['sair', 'exit']:
        break
    answer = chatbot(user_input)
    print(f"Chatbot: {answer}")

Inferencing Samples: 100%|██████████| 1/1 [00:01<00:00,  1.87s/ Batches]
/home/flavio/Dev/myenv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/flavio/Dev/myenv/lib/python3.12/site-packages/transformers/generation/configuration_utils.py:497: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(


Chatbot: Pergunta do usuário: qual principal linguagem utilizada para machine learning?

Contexto: Python

Resposta da IA:
1. A maioria dos computadores devem ser
